In [ ]:
!pip install zeugma
!pip install contractions==0.0.18
!pip install bs4
!pip install lxml
!python -m spacy download en_core_web_lg
!pip install contractions

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
STOPWORDS = set(stopwords.words('english'))
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import contractions
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))


import re, time
import pandas as pd
import numpy as np
import nltk
import spacy
import pickle
import contractions

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
from spacy.vectors import Vectors

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator

class GloveVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, CustomVectorizer,nlp):
        self.dim = 300
        self.CustomVectorizer=CustomVectorizer
        self.nlp=nlp

    def fit(self, X, y):
        return self

    def transform(self, X):
        embeddings=np.zeros((X.shape[0],self.dim))
        
        features = self.CustomVectorizer.get_feature_names_out()
        
        for i in range(0, X.shape[0]):
            feature_weights = X[i]
            more_than_zero_idx = np.nonzero(feature_weights > 0.0)
            
            feature_weights = feature_weights[more_than_zero_idx]
            present_features = features[more_than_zero_idx[1]]
            present_features_vecs = [
                self.nlp.vocab.get_vector(token) * (feature_weights[0, idx]) for idx, token in enumerate(present_features) if self.nlp.vocab.has_vector(token)
            ]
            
            if len(present_features_vecs) > 0:
                embeddings[i] = np.mean(present_features_vecs, axis=0)

        return embeddings

class CustomVectorizer(CountVectorizer):
    def __init__(self,lemmatize, stemming, keep_punctuation, keep_whitespace, merge_entities,sw_removal, nlp, **kwargs):
        super().__init__(**kwargs)
        self.lemmatize=lemmatize
        self.stemming=stemming
        self.keep_punctuation=keep_punctuation
        self.keep_whitespace=keep_whitespace
        self.merge_entities=merge_entities
        self.sw_removal=sw_removal
        self.sw=stopwords.words("english")
        self.nlp=nlp
    def tokenize(self,doc):
        # Lemmatizer/Tokenizer
        sp=self.nlp
        ps = PorterStemmer()

        # Clean text
        case_folded=doc.lower() # Case folding
        tags_removed = BeautifulSoup(case_folded, 'lxml').get_text() # Removed html tags
        doc_clean = contractions.fix(tags_removed) # Expand contractions

        # Merge entities
        # sp.add_pipe("merge_noun_chunks")
        # if self.merge_entities:
          # sp.add_pipe(sp.create_pipe('merge_entities'))
          # Restart kernel if changed
          # sp.add_pipe('merge_entities')
        
        # Tokenize
        tokens = sp(doc_clean)
        # Remove punctuation/whitespace
        if (not self.keep_punctuation) or (not self.keep_whitespace):
          tokens=[token for token in tokens if not ((token.is_punct and not self.keep_punctuation) or (token.is_space and not self.keep_whitespace))]
        
        if self.sw_removal:
            tokens = [token for token in tokens if token.text not in self.sw]
        
        # Lemmatization/Stemming
        if self.lemmatize:
          tokens = [token.lemma_ for token in tokens]
        elif self.stemming:
          tokens = [ps.stem(token.text) for token in tokens]
        else:
          tokens = [token.text for token in tokens]
        
        return tokens

    def build_analyzer(self):
        def analyser(doc):
            res=self.tokenize(doc)
            return(self._word_ngrams([token for token in res]))
        return(analyser)

class Preprocessor:
    """
    embedding(bool): Use glove embedding of length=25
    tf_scaling(bool): Use term-frequency scaling
    idf_scaling(bool): Use inverse document frequency scaling
    sw_removal(bool): Remove stop words
    lemmatize(bool): lemmatize
    stemming(bool): stem
    ngram_range(bool): values of n used when forming features
    topk(bool): Value of k using top k feature selection. Setting a value will indicate using feature selection
    keep_punctuation(bool): keep punctuation
    keep_whitespace(bool): keep whitespaces
    merge_entities(bool): Merge named entities e.g. "Empire State Building", "New York Times"
    topic_modelling(np.array): path to data_with_topicmodellingfeatures.csv file (in CS4248_G03 drive)
    """
    def __init__(self, embedding=False, tf_scaling=False, idf_scaling=False, sw_removal=False, 
                 lemmatize=False, stemming=False, ngram_range=(1,2), topk=None, keep_punctuation=False
                 ,keep_whitespace=False, merge_entities=False, topic_modelling=None):
        self.tf_scaling = tf_scaling
        self.idf_scaling = idf_scaling
        self.sw_removal = sw_removal
        self.lemmatize = lemmatize
        self.stemming=stemming
        self.topk = topk
        self.keep_punctuation=keep_punctuation
        self.embedding=embedding
        self.keep_whitespace=keep_whitespace
        self.merge_entities=merge_entities
        self.topic_modelling=topic_modelling
        if self.topic_modelling is not None:
            self.tm_feats=pd.read_csv(topic_modelling).iloc[: , -6:]
        self.nlp = spacy.load('en_core_web_lg')
        # print(self.nlp.pipe_names)

        if True:
            if lemmatize:
                self.nlp.add_pipe('lemmatizer')

            if not merge_entities:
                if not lemmatize:
                    self.nlp.disable_pipes('tagger', 'attribute_ruler')
                self.nlp.disable_pipes('parser', 'ner')

        # Initialize preprocessing pipeline objects
        self.count_vectorizer=('count vectorizer',CustomVectorizer(sw_removal=self.sw_removal, ngram_range=ngram_range,lemmatize=self.lemmatize, 
                                                                   stemming=self.stemming,keep_punctuation=self.keep_punctuation,
                                                                   keep_whitespace=self.keep_whitespace, merge_entities=self.merge_entities,
                                                                   nlp=self.nlp)) # Count
        self.k_best=('chi2score',SelectKBest(chi2,k=topk)) # topk
        self.tf_idf_transformer=('tf_transformer',TfidfTransformer(use_idf=idf_scaling)) #TF-IDF
        
        # Pipeline
        steps=[]
        steps.append(self.count_vectorizer)
        if not self.topk is None:
          steps.append(self.k_best)
        if self.tf_scaling:
          steps.append(self.tf_idf_transformer)
        if self.embedding:
          self.glove=('glove',GloveVectorTransformer(self.count_vectorizer[1],nlp=self.nlp))
          steps.append(self.glove)
        print(steps)
        self.model = Pipeline(steps)

    def fit(self, X, y):
        return self.model.fit(X, y)
    def transform(self, X):
        res=self.model.transform(X)
        if not self.topic_modelling is None:
            res=np.concatenate((res, self.tm_feats), axis=1)
        return res
    def fit_transform(self, X, y):
        res=self.model.fit_transform(X,y)
        if not self.topic_modelling is None:
            print(self.tm_feats)
            res=np.concatenate((res, self.tm_feats), axis=1)
        return res

In [ ]:
# Edit if running locally
FULLTRAIN_PATH = 'fulltrain.csv'
BALANCED_TEST_PATH = 'balancedtest.csv'

# Loading data from csv file
data = pd.read_csv(FULLTRAIN_PATH, header=None)
X_train = data[1]
Y_train = data[0]

# Same tokenization transforms for test.
test_data = pd.read_csv(BALANCED_TEST_PATH, header=None)
X_test = test_data[1]
Y_test = test_data[0]

p = Preprocessor(
    embedding=False,
    tf_scaling=True,
    idf_scaling=True,
    sw_removal=True,
    lemmatize=True,
    stemming=False,
    ngram_range=(1,1),
    topk=20000,         # 20000
    keep_punctuation=False,
    keep_whitespace=False,
    merge_entities=False,
    binary=False
)
print(p.nlp.pipeline)

In [ ]:
Y_train_pred = None
Y_test_pred = None

train_X_vec = p.fit_transform(X_train, Y_train)
# train_X_vec = p.fit_transform(X_test, Y_test)
test_X_vec = p.transform(X_test)

if not isinstance(train_X_vec, np.ndarray):
    train_X_vec = train_X_vec.toarray()
if not isinstance(test_X_vec, np.ndarray):
    test_X_vec = test_X_vec.toarray()

In [ ]:
print('Starting models...')
train_X_vec.shape  # verify shape

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
def evaluate(X, Y):
    print(f'Accuracy: {accuracy_score(Y, X)}')
    print(f'Precision: {precision_score(Y, X, average=None)}')
    print(f'Recall: {recall_score(Y, X, average=None)}')
    print(f'F1: {f1_score(Y, X, average=None)}')

In [ ]:
# Logistic Regression
model = LogisticRegression()

# OVR
# model = LogisticRegression(multi_class='ovr', solver='liblinear')

# OVO
# model = OneVsOneClassifier(LinearSVC(random_state=0))

In [ ]:
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

In [ ]:
clf = GridSearchCV(model, param_grid = param_grid, cv = 3, verbose=True, n_jobs=None)

In [ ]:
best_clf = clf.fit(train_X_vec,Y_train)

In [ ]:
best_clf.best_estimator_

In [ ]:
Y_train_pred = best_clf.predict(train_X_vec)
Y_test_pred = best_clf.predict(test_X_vec)

In [ ]:
evaluate(Y_train_pred, Y_train)
evaluate(Y_test_pred, Y_test)